# DATABASE TABLE MAKE

In [1]:
import pandas as pd

In [2]:
import pandas as pd

df_list = []
for sheet_num in range(5):
    df_dict = {}
    df = pd.read_excel('2023_BDC_3차년도 테이블정의서_Nick.xlsx', sheet_name=sheet_num)
    for key, value in zip(df["표준컬럼 영문명"], df["표준 데이터타입"]):
        df_dict[key] = value
    df_list.append([sheet_num, df_dict])

In [1]:
import os
import sys
import pandas as pd
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname("ETL"))))

In [4]:
from ETL import EL

ModuleNotFoundError: No module named 'ETL'

In [22]:
load = EL.DataLoad()

In [23]:
load.Login("postgres", "1234", "211.245.172.58", "5432", "nia_test")

In [24]:
import psycopg2
conn = psycopg2.connect(load.url)

In [25]:
cur = conn.cursor()
for idx in range(len(df_list)):
    table_name = f"tb_hb_kor_test_{idx}"
    column_data_types = df_list[idx][1]
    columns = ', '.join([f'{column} {data_type}' for column, data_type in column_data_types.items()])
    create_table_query = f"CREATE TABLE {table_name} ({columns})"
    cur.execute(create_table_query)

    # 커밋
    conn.commit()

# DATABASE TABLE DELETE

In [26]:
for idx in range(len(df_list)):
    table_name = f"tb_hb_kor_test_{idx}"
    drop_table_query = f"DROP TABLE IF EXISTS {table_name}"

    # 테이블 삭제
    cur.execute(drop_table_query)

    # 커밋
    conn.commit()

# DATA HANDLING

In [29]:
import pandas as pd
from tqdm import tqdm
import re

In [30]:
df = pd.read_excel('2023_BDC_3차년도 테이블정의서_Nick.xlsx', sheet_name=1)

In [31]:
def make_ev_dict(df) -> pd.DataFrame:
    ev_dict = {}
    for key, value in zip(df["표준컬럼 영문명"], df["표준 데이터타입"]):
        ev_dict[key] = value
    columns = [x.lower() for x in df["표준컬럼 영문명"]]
    return ev_dict, pd.DataFrame(columns=columns)

In [32]:
korea_ev_info = make_ev_dict(pd.read_excel('2023_BDC_3차년도 테이블정의서_Nick.xlsx', sheet_name=1))[0]
korea_ev_info_df = make_ev_dict(pd.read_excel('2023_BDC_3차년도 테이블정의서_Nick.xlsx', sheet_name=1))[1]
korea_ev_finan = make_ev_dict(pd.read_excel('2023_BDC_3차년도 테이블정의서_Nick.xlsx', sheet_name=0))[0]
korea_ev_finan_df = make_ev_dict(pd.read_excel('2023_BDC_3차년도 테이블정의서_Nick.xlsx', sheet_name=0))[1]
global_ev_info = make_ev_dict(pd.read_excel('2023_BDC_3차년도 테이블정의서_Nick.xlsx', sheet_name=3))[0]
global_ev_info_df = make_ev_dict(pd.read_excel('2023_BDC_3차년도 테이블정의서_Nick.xlsx', sheet_name=3))[1]
global_ev_finan = make_ev_dict(pd.read_excel('2023_BDC_3차년도 테이블정의서_Nick.xlsx', sheet_name=2))[0]
global_ev_finan_df = make_ev_dict(pd.read_excel('2023_BDC_3차년도 테이블정의서_Nick.xlsx', sheet_name=2))[1]
third_info = make_ev_dict(pd.read_excel('2023_BDC_3차년도 테이블정의서_Nick.xlsx', sheet_name=4))[0]
third_info_df = make_ev_dict(pd.read_excel('2023_BDC_3차년도 테이블정의서_Nick.xlsx', sheet_name=4))[1]

In [33]:
respond_df = pd.read_excel("3차년도_대응표.xlsx")

In [34]:
import os
second_finan_lst = "C:/Users/user/Desktop/유신영 업무폴더/BDC/1_BDC 데이터 정리/13_2023년 3분기/131_2023년 3분기 3차년도/2023년 3분기 3차년도 재무정보/데이터베이스 적재/"
finan_lst = os.listdir(second_finan_lst)

In [35]:
append_base_df = pd.read_excel(second_finan_lst + "tb_hb_kor_plcfi_d.xlsx")

In [36]:
filtered_df = respond_df[["2차년도_한국_재무_영문", "3차년도_한국_재무_영문"]].dropna()
append_df = append_base_df[filtered_df["2차년도_한국_재무_영문"]]
filtered_dict = {}
for key, value in zip(filtered_df["2차년도_한국_재무_영문"], filtered_df["3차년도_한국_재무_영문"]):
    filtered_dict[key] = value.lower()
append_base_df = append_df.rename(columns= filtered_dict)
# korea_ev_finan_df = korea_ev_finan_df.append(append_df)
# korea_ev_finan_df = korea_ev_finan_df.reset_index(drop=True)

# INFORMATION DF TRANSFORM

In [10]:
info_korea = pd.read_excel("한국_3차년도_재무정보_일반.xlsx")
info_korea = info_korea[info_korea["status"] == 0]
respond_df_info = respond_df[["DART_한국_일반정보", "DART_한국_대응정보"]].dropna()
info_korea = info_korea[respond_df_info["DART_한국_일반정보"]]
info_korea_dict = {}
for key, value in zip(respond_df_info["DART_한국_일반정보"], respond_df_info["DART_한국_대응정보"]):
    info_korea_dict[key] = value.lower()
info_korea = info_korea.rename(columns=info_korea_dict)
info_korea = info_korea.reset_index(drop=True)
for idx in range(len(info_korea["lstng_cd"])):
    info_korea["lstng_cd"][idx] = int(info_korea["lstng_cd"][idx])

C:\Users\user\AppData\Local\Temp\ipykernel_31920\2205498205.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info_korea["lstng_cd"][idx] = int(info_korea["lstng_cd"][idx])


In [11]:
for idx in range(len(korea_ev_finan_df)):
    symbol = korea_ev_finan_df["lstng_cd"][idx]
    append_df = info_korea[info_korea["lstng_cd"] == symbol]
    if not append_df.empty:
        korea_ev_finan_df.loc[idx, list(append_df.columns)] = append_df.values[0]

In [12]:
korea_ev_finan_df.to_excel("test.xlsx", index=False)

In [38]:
korea_stock = pd.read_csv("C:/Users/user/Desktop/유신영 업무폴더/BDC/1_BDC 데이터 정리/13_2023년 3분기/131_2023년 3분기 3차년도/2023년 3분기 3차년도 시가총액정보/가공후/korea/korea.csv")

In [39]:
korea_stock_dict = {"Date":"stndd_dt",
                    "Code":"lstng_cd",
                    "Open":"stock_start",
                    "Close":"stock_end",
                    "High":"stock_high",
                    "Low":"stock_low",
                    "Marcap":"mrprc_ttamt"}

In [40]:
korea_stock = korea_stock.rename(columns= korea_stock_dict)

In [41]:
korea_stock["stock_avg"] = (korea_stock["stock_high"] + korea_stock["stock_low"]) // 2

In [17]:
korea_stock = korea_stock[["stndd_dt", "lstng_cd", "stock_start", "stock_high", "stock_low", "stock_end", "stock_avg", "mrprc_ttamt"]]

In [44]:
symbol_lst = [int(x) for x in set(korea_stock["lstng_cd"]) if not re.search(r"[a-zA-Z]", x)]

In [45]:
duplicated_symbol = set(list(symbol_lst)) & set(list(info_korea["lstng_cd"]))

In [46]:
duplicated_symbol

{100090.0,
 100120.0,
 100130.0,
 100220.0,
 100250.0,
 100700.0,
 100840.0,
 101000.0,
 101160.0,
 101170.0,
 101360.0,
 101400.0,
 101490.0,
 101530.0,
 101670.0,
 101730.0,
 101930.0,
 102120.0,
 102260.0,
 102280.0,
 102370.0,
 102460.0,
 102710.0,
 102940.0,
 103140.0,
 103590.0,
 104460.0,
 104540.0,
 104620.0,
 104830.0,
 105330.0,
 105630.0,
 105740.0,
 106080.0,
 106240.0,
 106520.0,
 107590.0,
 108230.0,
 108320.0,
 108380.0,
 108490.0,
 108670.0,
 108860.0,
 109070.0,
 109860.0,
 109960.0,
 110790.0,
 110990.0,
 111110.0,
 111770.0,
 111870.0,
 112040.0,
 112610.0,
 113810.0,
 114090.0,
 114120.0,
 114190.0,
 114630.0,
 114810.0,
 115180.0,
 115310.0,
 115390.0,
 115450.0,
 115480.0,
 115530.0,
 115570.0,
 117580.0,
 117670.0,
 117730.0,
 118000.0,
 118990.0,
 119610.0,
 119650.0,
 119850.0,
 119860.0,
 120110.0,
 121440.0,
 121600.0,
 121800.0,
 121850.0,
 122310.0,
 122350.0,
 122450.0,
 122640.0,
 122870.0,
 122900.0,
 122990.0,
 123040.0,
 123330.0,
 123420.0,
 123570.0,

In [47]:
symbols_ = [str(int(i)) for i in duplicated_symbol]

In [49]:
df_A = korea_stock[korea_stock['lstng_cd'].isin(symbols_)]

In [51]:
df_A["lstng_cd"] = df_A["lstng_cd"].astype(int)

C:\Users\user\AppData\Local\Temp\ipykernel_31920\3751442669.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_A["lstng_cd"] = df_A["lstng_cd"].astype(int)


In [52]:
df_A

,stndd_dt,lstng_cd,Name,stock_start,stock_high,stock_low,stock_end,Stocks,mrprc_ttamt,stock_avg
9,2018-01-02,207940,삼성바이오로직스,367000,391000,367000,389500,66165000,25771267500000,379000
43,2018-01-02,139480,이마트,272000,273000,260500,262000,27875819,7303464578000,266750
46,2018-01-02,215600,신라젠,95900,104000,93300,102500,68001447,6970148317500,98650
48,2018-01-02,161390,한국타이어,54600,55400,54600,55000,123875069,6813128795000,55000
49,2018-01-02,267250,현대로보틱스,387500,414500,387000,414500,16286617,6750802746500,400750
...,...,...,...,...,...,...,...,...,...,...
2931984,2022-12-29,286750,나노브릭,1330,1340,1295,1325,20464549,27115527425,1317
2931986,2022-12-29,290660,네오펙트,1435,1475,1335,1395,19240617,26840660715,1405
2932026,2022-12-29,174880,장원테크,928,931,836,878,21906325,19233753350,883
2932037,2022-12-29,204630,스튜디오산타클로스,1820,1820,1645,1680,10850308,18228517440,1732


In [54]:
korea_ev_finan_df = korea_ev_finan_df.append(df_A)

C:\Users\user\AppData\Local\Temp\ipykernel_31920\73722823.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  korea_ev_finan_df = korea_ev_finan_df.append(df_A)


In [55]:
korea_ev_finan_df.reset_index(inplace=True)

In [57]:
for idx in tqdm(range(len(korea_ev_finan_df))):
    symbol = korea_ev_finan_df["lstng_cd"][idx]
    append_df = info_korea[info_korea["lstng_cd"] == symbol]
    if not append_df.empty:
        korea_ev_finan_df.loc[idx, list(append_df.columns)] = append_df.values[0]

  0%|          | 0/545466 [00:00<?, ?it/s]

100%|██████████| 545466/545466 [3:16:56<00:00, 46.16it/s]   


In [60]:
korea_ev_finan_df.reset_index(inplace=True)

In [62]:
for idx in tqdm(range(len(korea_ev_finan_df))):
    symbol = korea_ev_finan_df["lstng_cd"][idx]
    append_df = append_base_df[append_base_df["lstng_cd"] == symbol]
    if not append_df.empty:
        korea_ev_finan_df.loc[idx, list(append_df.columns)] = append_df.values[0]

100%|██████████| 545466/545466 [26:41:11<00:00,  5.68it/s]    


In [66]:
korea_ev_finan_df = korea_ev_finan_df.drop(["level_0", "index"], axis=1)

In [72]:
korea_ev_finan_df["hb_ntn_cd"] = "KOR"
korea_ev_finan_df["kr_ntn_nm"] = "한국"
korea_ev_finan_df["engls_ntn_nm"] = "KOREA"
korea_ev_finan_df["ntn_lngg_cd_val"] = "KOR"
korea_ev_finan_df["kr_lngg_nm"] = "한국어"
korea_ev_finan_df["engls_lngg_nm"] = "KOREAN"

In [74]:
korea_ev_finan_df = korea_ev_finan_df.drop(["Name", "Stocks"], axis=1)

In [79]:
korea_ev_finan_df.to_parquet("대한민국_3차_재무.parquet", index=False)

In [117]:
korea_ev_finan_df["lstng_cd"] = korea_ev_finan_df["lstng_cd"].astype(int)

In [125]:
korea_ev_finan_df["lstng_cd"][idx]

161570

In [126]:
append_df[append_df["lstng_cd"] == symbol]

,keyval,hb_ntn_cd,acplc_lngg_entrp_nm,engls_entrp_nm,crprt_rgno,bsnsm_rgno,fndtn_dt,stock_mrkt_cd,engls_stock_mrkt_nm,lstng_dt,...,nncrrnt_debt_amt,debt_summ,captl_summ,debt_captl_summ_amt,bsn_acti_csflw,invsm_acti_cash_infl_amt,fnnr_acti_cash_infl_amt,cash_incre_amt,bsis_cash_amt,entrm_cash_amt


In [91]:
korea_ev_finan_df = korea_ev_finan_df.drop(['level_0', 'index'], axis=1)

In [93]:
korea_ev_finan_df.to_excel("korea.xlsx", index=False)

In [109]:
append_df["debt_summ"].isnull().value_counts()

False    4525
True       15
Name: debt_summ, dtype: int64

In [122]:
korea_ev_finan_df["debt_summ"].isnull().value_counts()

True     545465
False         1
Name: debt_summ, dtype: int64